In [1]:
import cudaq
from cudaq import spin

import matplotlib.pyplot as plt

import numpy as np

import torch
from torch.autograd import Function
from torchvision import datasets, transforms
import torch.optim as optim
import torch.nn as nn
import torchvision

from sklearn.model_selection import train_test_split

torch.manual_seed(22)
cudaq.set_random_seed(44)

# device = torch.device('cpu')
# cudaq.set_target("qpp-cpu")

cudaq.set_target("nvidia") # nvidia-mqpu
device = torch.device("cuda")

print(cudaq.num_available_gpus())

8


In [ ]:
class QuantumFunction(Function):
    """Allows the quantum circuit to input data, output expectation values
    and calculate gradients of variational parameters via finite difference"""

    def __init__(self, qubit_count: int, hamiltonian: cudaq.SpinOperator):
        """Define the quantum circuit in CUDA Quantum"""

        @cudaq.kernel
        def kernel(qubit_count: int, thetas: np.ndarray):
            qubits = cudaq.qvector(qubit_count)
            ry(thetas[0], qubits[0])
            rx(thetas[1], qubits[0])

        self.kernel = kernel
        self.qubit_count = qubit_count
        self.hamiltonian = hamiltonian

    def run(self, theta_vals: torch.tensor) -> torch.tensor:
        """Excetute the quantum circuit to output an expectation value"""

        #If running on GPU, thetas is a torch.tensor that will live on GPU memory. The observe function calls a .tolist() method on inputs which moves thetas from GPU to CPU.

        qubit_count = [self.qubit_count for _ in range(theta_vals.shape[0])]

        results = cudaq.observe(self.kernel, self.hamiltonian, qubit_count,
                                theta_vals)

        exp_vals = [results[i].expectation() for i in range(len(results))]
        exp_vals = torch.Tensor(exp_vals).to(device)

        return exp_vals

    @staticmethod
    def forward(ctx, thetas: torch.tensor, quantum_circuit,
                shift) -> torch.tensor:

        # Save shift and quantum_circuit in context to use in backward.
        ctx.shift = shift
        ctx.quantum_circuit = quantum_circuit

        # Calculate expectation value.
        exp_vals = ctx.quantum_circuit.run(thetas).reshape(-1, 1)

        ctx.save_for_backward(thetas, exp_vals)

        return exp_vals

    @staticmethod
    def backward(ctx, grad_output):
        """Backward pass computation via finite difference"""

        thetas, _ = ctx.saved_tensors

        gradients = torch.zeros(thetas.shape, device=device)

        for i in range(thetas.shape[1]):

            thetas_plus = thetas.clone()
            thetas_plus[:, i] += ctx.shift
            exp_vals_plus = ctx.quantum_circuit.run(thetas_plus)

            thetas_minus = thetas.clone()
            thetas_minus[:, i] -= ctx.shift
            exp_vals_minus = ctx.quantum_circuit.run(thetas_minus)

            gradients[:, i] = (exp_vals_plus - exp_vals_minus) / (2 * ctx.shift)

        gradients = torch.mul(grad_output, gradients)

        return gradients, None, None

In [14]:
import numpy as np
# Define our kernel.
@cudaq.kernel
def generator_kernel(qubit_count: int, thetas: np.ndarray, layers: int):
    # Allocate our qubits.
    qubits = cudaq.qvector(qubit_count)
    theta_count = 0
    for _ in range(layers):
        for ix in range(qubit_count):
            ry(thetas[theta_count], qubits[ix])
            theta_count += 1
            rx(thetas[theta_count], qubits[ix])
            theta_count += 1
            rz(thetas[theta_count], qubits[ix])
            theta_count += 1
        for ix in range(qubit_count - 1):
            x.ctrl(qubits[ix], qubits[ix + 1])
    # mz(qubits)

qubit_count = 2
layers = 3
thetas = np.random.normal(0, 1, size=[layers, qubit_count*3])
print(thetas)

# print(cudaq.draw(generator_kernel, qubit_count, thetas.reshape(-1), layers))D
qstate = cudaq.get_state(generator_kernel, qubit_count, thetas.reshape(-1), layers)
print(qstate)
print(type(qstate))
a = np.array(qstate)
print(a)
print(np.abs(a))
print(np.sum(np.abs(a)))

[[ 0.05565613 -1.56922158  0.00834206  1.75765741 -1.76210499  1.83760909]
 [-0.55921046 -0.37518948  1.10335608 -1.03629477  0.99023608  0.84001287]
 [-1.21844592  0.26831482 -0.3886333   1.22784802 -1.56450877  1.05183222]]
0.0254798-0.0952863j -0.611035-0.346426j -0.424929+0.460481j -0.216561+0.239562j 

<class 'cudaq.mlir._mlir_libs._quakeDialects.cudaq_runtime.State'>
[ 0.02547981-0.09528627j -0.61103505-0.34642565j -0.42492929+0.46048146j
 -0.21656093+0.23956227j]
[0.09863414 0.70240627 0.62658446 0.32293764]
1.7505624998591665


In [8]:
qstate.amplitudes()

AttributeError: 'cudaq.mlir._mlir_libs._quakeDialects.cudaq_runtime' object has no attribute 'amplitudes'

In [148]:
# qstate_2 = cudaq.mlir._mlir_libs._quakeDialects.cudaq_runtime.get_state(generator_kernel, qubit_count, thetas.reshape(-1), layers)
orthogonal_state = np.zeros(2**qubit_count, dtype=np.complex128)
orthogonal_state[1] = 1
qstate_2 = cudaq.mlir._mlir_libs._quakeDialects.cudaq_runtime.State(orthogonal_state)
cudaq.mlir._mlir_libs._quakeDialects.cudaq_runtime.State.overlap(qstate_2,qstate_2)
# qstate_2.dump()

0.0

In [3]:
for i in range(2**qubit_count):
    orthogonal_state = np.zeros(2**qubit_count, dtype=np.complex128)
    orthogonal_state[i] = 1
    print(qstate.overlap((orthogonal_state)))

0.2768849878496445
0.6842258617831293
0.31856297904373315
0.5947163379547546


In [5]:
c = np.array([1, 2j, 3, 4j, 5, 6j, 7, 8j], dtype=np.complex64)
state = cudaq.State.State.from_data(c)
# state = cudaq.State.from_data(c)

cudaq.State

AttributeError: type object 'cudaq.mlir._mlir_libs._quakeDialects.cudaq_runtime' has no attribute 'State'

In [100]:
s2 = cudaq.State(state)
s2.ToTensor()

AttributeError: 'cudaq.mlir._mlir_libs._quakeDialects.cudaq_runtime' object has no attribute 'ToTensor'

In [79]:
state.

TypeError: object of type 'cudaq.mlir._mlir_libs._quakeDialects.cudaq_runtime.State' has no len()

In [75]:
@cudaq.kernel
def generator_kernel(qubit_count: int, thetas: np.ndarray, layers: int):
    # Allocate our qubits.
    qubits = cudaq.qvector(qubit_count)
    theta_count = 0
    for _ in range(layers):
        for ix in range(qubit_count):
            ry(thetas[theta_count], qubits[ix])
            theta_count += 1
            rx(thetas[theta_count], qubits[ix])
            theta_count += 1
        for ix in range(qubit_count - 1):
            x.ctrl(qubits[ix], qubits[ix + 1])
    mz(qubits)

qubit_count = 5
layers = 3
thetas = np.random.normal(0, 1, size=[layers, qubit_count*2])
print(thetas)



[[-0.29118993 -1.21485278  0.1925102  -0.40727453 -0.11457405  0.96741856
   1.00384274  0.87515969 -1.30872092 -0.13819804]
 [ 0.38545717  0.40496696 -1.26578799 -0.24007079 -1.34706637  0.51291888
  -1.02981805 -0.71936201 -0.09823871  0.20776848]
 [-0.51931479 -0.34918431  0.44556586 -0.21426969  0.6627725   1.67861425
   0.5048656   1.2231496  -1.27188848 -0.45908467]]
0+0j 0+0j 0+0j 0+0j 0+0j 0+0j 0+0j 0+0j 0+0j 0+0j 0+0j 0+0j 0+0j 0+0j 0+0j 0+0j 0+0j -0.976997+0.213255j 0+0j 0+0j 0+0j 0+0j 0+0j 0+0j 0+0j 0+0j 0+0j 0+0j 0+0j 0+0j 0+0j 0+0j 



In [74]:
import numpy as np

N = 3
state = np.zeros(2**N, dtype=complex)
state[7] = 1. 
print(state)
   

kernel = cudaq.from_state(state)
#extracted_state = cudaq.get_state(kernel) 
#print(extracted_state)
result = cudaq.sample(kernel)
print(result) # you get |110> instead of |111>

[0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 1.+0.j]


AttributeError: module 'cudaq' has no attribute 'from_state'

In [62]:
sample_result = cudaq.sample(generator_kernel, qubit_count, thetas.reshape(-1), layers)
print(f"most probable = {sample_result.most_probable()}")
print(f"expectation_value = {sample_result.expectation()}")
print(sample_result)

most probable = 00010
expectation_value = -0.09399999999999997
{ 11111:47 01111:12 01010:11 01110:10 01101:63 10010:80 00010:173 11010:37 00100:21 00110:42 10101:16 11000:42 01000:16 00111:70 11100:6 10111:18 10000:6 11011:25 10100:8 10110:5 11110:36 00001:25 10001:7 00000:23 01001:8 11101:38 11001:12 00101:19 01011:9 00011:60 01100:24 10011:31 }



In [24]:
@cudaq.kernel
def test_kernel(qubit_count: int, thetas: np.ndarray, layers: int):

    qubits = cudaq.qvector(qubit_count)
    for il in range(layers):
        for ix in range(qubit_count):
            ry(thetas[il][ix], qubits[il][ix])
        # rx(thetas[ix], qubits[ix])
print(cudaq.draw(test_kernel, 2, np.zeros((2,2)), 2))

CompilerError: 696875955.py:7: [91merror: [0m[1munhandled subscript
	 (offending source -> thetas[il][ix])[0m

In [18]:
np.random.normal(0, 1, size=[2])

array([-0.66339776,  0.45141048])

In [8]:
qubit_count = 5
layers = 3
thetas = np.random.normal(0, 1, size=[layers, qubit_count*2])
thetas[0][0]

-0.07284855593608636